In [1]:
from quantum_field import *
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
import pandas as pd
import numpy as np
import datashader as ds
from colorcet import fire, kbc, bmw, gray
template="plotly_dark"

In [2]:
# Construct the slits
wallMass = 10
yWall = 10
slitAperture = 100
leftEdge = Object(-slitAperture / 2, yWall, wallMass)
leftWall = [Object(-100 - slitAperture / 2, yWall, wallMass), leftEdge]
rightEdge = Object(slitAperture / 2, yWall, wallMass)
rightWall = [Object(100 + slitAperture / 2, yWall, wallMass), rightEdge]
edges = [leftEdge, rightEdge]
walls = [leftWall, rightWall]

# Uncomment for one slit
# edges = [Object(i+slitAperture/2, yWall, wallMass) for i in range(10)]
edges = [rightEdge]
walls = [edges]


# Create the particles
v_y_particles = 300
particle_y0_step = 100
particle_x0_fraction = 1

particles = [
    Particle(x_t0=i/particle_x0_fraction, y_t0=j, v_x=0, v_y=v_y_particles, mass=1) for j in range(-100, 0, particle_y0_step)
    for i in range(0, (slitAperture//2)*particle_x0_fraction-1, 1)
]


# Define parameters
dt = 1
cos_sin = "none"
power = 1
r_modifier = 1
G=100
c=v_y_particles
gravityCutoff = 1000
yGrid = True

# Punch it
for t in range(100):
    for particle in particles:
        if particle.y > yWall*2:
            dt = 1
        if particle.v_y > 0:
            updateParticle(
                particle,
                edges,
                G=G,
                dt=dt,
                r_modifier=r_modifier,
                c=c,
                power=power,
                gravityCutoff=gravityCutoff,
                cos_sin=cos_sin,
                yGrid=yGrid,
            )

In [3]:
print(particles[0].v_y)
print(particles[0].v_x)
print(particles[0].x_trail)
print(particles[0].y_trail)


311.5205833637597
14.355974916684668
[0.00000000e+00 0.00000000e+00 9.44559258e-05 2.85493267e-04
 5.75298329e-04 9.66120288e-04 1.46027332e-03 2.06013879e-03
 2.76816759e-03 3.58688248e-03 4.51888055e-03 5.56683581e-03
 6.73350183e-03 8.02171449e-03 9.43439487e-03 1.09745522e-02
 1.26452869e-02 1.44497938e-02 1.63913656e-02 1.84733960e-02
 2.06993836e-02 2.30729353e-02 2.55977705e-02 2.82777247e-02
 3.11167537e-02 3.41189383e-02 3.72884881e-02 4.06297465e-02
 4.41471954e-02 4.78454602e-02 5.17293146e-02 5.58036863e-02
 6.00736620e-02 6.45444936e-02 6.92216037e-02 7.41105916e-02
 7.92172399e-02 8.45475206e-02 9.01076019e-02 9.59038552e-02
 1.01942862e-01 1.08231421e-01 1.14776555e-01 1.21585523e-01
 1.28665819e-01 1.36025191e-01 1.43671640e-01 1.51613435e-01
 1.59859118e-01 1.68417514e-01 1.77297742e-01 1.86509219e-01
 1.96061675e-01 2.05965159e-01 2.16230051e-01 2.26867068e-01
 2.37887277e-01 2.49302104e-01 2.61123340e-01 2.73363156e-01
 2.86034109e-01 2.99149151e-01 3.12721639e-01 3.

In [4]:
fig = go.Figure(
    data=[
        go.Scatter(
            x=particle.x_trail,
            y=particle.y_trail,
            mode="markers+lines",
            marker_size=5,
            marker_color="crimson",
            line_width=1,
        )
        for particle in particles
    ]
    + [
        go.Scatter(
            x=[particle.x for particle in particles],
            y=[particle.y for particle in particles],
            mode="markers",
            marker_color="red",
            marker_size=2,
            line_width=1,
        )
    ]
    + [
        go.Scatter(
            x=[edge.x for edge in wall],
            y=[edge.y for edge in wall],
            mode="markers+lines",
            marker_color="darkgray",
        )
        for wall in walls
    ],
    layout=dict(width=1600, height=800),
)
fig.update_layout(template=template)
fig.write_html(
    f"plotly_graphs/_test_2d_{len(particles)}p_ygrid{yGrid}_apert{slitAperture}_fract{particle_x0_fraction}_{cos_sin}{power}_rmod{r_modifier}_G{G}_c{c}_y0_step{particle_y0_step}_gravCut{gravityCutoff}.html"
)
fig.show()


In [5]:
canvas = ds.Canvas(plot_width=1600, plot_height=800)
points = pd.DataFrame(np.vstack((np.hstack([particle.x_trail for particle in particles]), np.hstack([particle.y_trail for particle in particles])))).T
points.rename(columns={0: "x", 1: "y"}, inplace=True)
shader = canvas.points(points, x="x", y="y")
coordinates_x = shader.coords["x"].values
coordinates_y = shader.coords["y"].values
img = ds.transfer_functions.shade(shader, cmap=fire, how='eq_hist', alpha=255)[::-1].to_pil()

In [6]:
fig = go.Figure(
    data=[
        go.Scatter(
            x=[particle.x for particle in particles],
            y=[particle.y for particle in particles],
            mode="markers",
            marker_color="red",
            marker_size=2,
            line_width=1,
        )
    ]
    + [
        go.Scatter(
            x=[edge.x for edge in wall],
            y=[edge.y for edge in wall],
            mode="markers+lines",
            marker_color="darkgray",
        )
        for wall in walls
    ],
    layout=dict(width=1600, height=800),
)
fig.add_layout_image(
        dict(
            source=img,
            xref="x",
            yref="y",
            x=20,
            y=50,
            sizex=200,
            sizey=200,
            sizing="stretch",
            opacity=1,
            layer="above")
)
fig.update_layout(template=template)
fig.write_html(f"plotly_graphs/test_{len(particles)}particles_aperture{slitAperture}_fraction{particle_x0_fraction}_{cos_sin}{power}_rmodifier{r_modifier}_G{G}_c{c}_y0_step{particle_y0_step}.html")
fig.show()